In [73]:
#data import paths

import os
import pandas as pd
import numpy as np

if os.name == 'nt':
    windows = True
elif os.name == 'posix':
    windows = False

if windows:
    grip_ngrip_depths = "E:/GitHub/BICC/Paleochrono/BICC2025/raw data/WDC-NGRIP Link Interpolation/GICC_AgeLinks.xls"
    grip_wdc_links = "E:/GitHub/BICC/Paleochrono/BICC2025/raw data/WDC-NGRIP Link Interpolation/GRIP_WDC_TephraLinks.xlsx"
else:
    pass

In [74]:
grip_ngrip_depths = pd.read_excel(grip_ngrip_depths, comment="#", skiprows=5, sheet_name=0, usecols=[2,3,4], names=["NGRIP1_Depth", "NGRIP2_Depth", "GRIP_Depth"])

ngrip1_grip_links = grip_ngrip_depths[["NGRIP1_Depth", "GRIP_Depth"]]
ngrip1_grip_links = ngrip1_grip_links.dropna()

ngrip2_grip_links = grip_ngrip_depths[["NGRIP2_Depth", "GRIP_Depth"]]
ngrip2_grip_links = ngrip2_grip_links.dropna()

grip_wdc_links = pd.read_excel(grip_wdc_links, comment="#", skiprows=29, sheet_name=0, usecols=[0,1], names=["WDC_Depth", "GRIP_Depth"])
grip_wdc_links = grip_wdc_links.dropna()

In [ ]:
ngrip1_wdc_links = np.interp(grip_wdc_links["GRIP_Depth"], ngrip1_grip_links["GRIP_Depth"], ngrip1_grip_links["NGRIP1_Depth"])
ngrip2_wdc_links = np.interp(grip_wdc_links["GRIP_Depth"], ngrip2_grip_links["GRIP_Depth"], ngrip2_grip_links["NGRIP2_Depth"])

#trim, this is done by hand but could actually make it auto if its important
ngrip1_wdc_links[77:] = np.nan
ngrip2_wdc_links[0:76] = np.nan

export = {
    "#GRIP_Depth": grip_wdc_links["GRIP_Depth"],
    "WDC_Depth": grip_wdc_links["WDC_Depth"],
    "NGRIP1_Depth": ngrip1_wdc_links,
    "NGRIP2_Depth": ngrip2_wdc_links
}

export = pd.DataFrame(export)
export.to_excel("E:/GitHub/BICC/Paleochrono/BICC2025/raw data/WDC-NGRIP Link Interpolation/EXPORT_GRIP_WDC_NGRIP_Links.xlsx", index=False)

In [ ]:
##new Jul 16 2025 for multi-NGRIP

In [32]:
#load the data

import os
import pandas as pd
import numpy as np

os.chdir('/Users/quinnmackay/Documents/GitHub/paleochrono/BICC2025/raw data/WDC-NGRIP Link Interpolation/')

gicc05_scale = pd.read_csv('GICC05 data/datasets/GICC05_time_scale.tab', skiprows=40, delimiter='\t')

gicc21_scale = pd.read_excel('GICC05-GICC21 Conversion.xlsx', sheet_name=1, skiprows=1, 
                             names=['Age b2k', 'dt yrs', 'yrs CE/BCE', 'EGRIP m', 'NEEM m', 
                                    'NGRIP1 m', 'NGRIP2 m', 'NEEM2011-S1 m', 'GRIP m', 
                                    'DYE3-79 m', 'DYE3-4B m', 'DYE3-18C m'])

grip_wdc_tephra = pd.read_excel('GRIP_WDC_TephraLinks.xlsx', skiprows=29, names=['WDC m', 'GRIP m', "WD2014 age iso", 'WD2014 age yr bp 1950', 'GICC05 age', 'age diff yr', 'GRIP dz/dt', 'distance', 'type'], usecols = range(9))



NGRIP1 GICC21 until 730 (159.345), NGRIP2 GICC21 until 3245 (582.400)

NGRIP1 GICC05 until 9527 (1346.87), NGRIP2 GICC05 until end

In [80]:
#get interpolates

gicc05_ngrip1 = np.interp(grip_wdc_tephra['GRIP m'], gicc05_scale['Depth ice/snow [m] (GRIP)'], gicc05_scale['Depth ice/snow [m] (NGRIP1)'])

gicc05_ngrip2 = np.interp(grip_wdc_tephra['GRIP m'], gicc05_scale['Depth ice/snow [m] (GRIP)'], gicc05_scale['Depth ice/snow [m] (NGRIP2)'])

gicc21_ngrip1 = np.interp(grip_wdc_tephra['GRIP m'], gicc21_scale['GRIP m'], gicc21_scale['NGRIP1 m'])

gicc21_ngrip2 = np.interp(grip_wdc_tephra['GRIP m'], gicc21_scale['GRIP m'], gicc21_scale['NGRIP2 m'])

#create match array

ngrip_match_points = []

#not super accurate but will have to do
for val in gicc21_ngrip1:
    if val <= 159.345:
        ngrip_match_points.append(val)

for val in gicc21_ngrip2:
    if val > 159.345 and val <= 582.400:
        ngrip_match_points.append(val)

for val in gicc05_ngrip1:
    if val > 582.400 and val <= 1346.87:
        ngrip_match_points.append(val)

for val in gicc05_ngrip2:
    if val > 1346.87:
        ngrip_match_points.append(val)

uncertainty_col = 20*np.ones(len(ngrip_match_points))

for i, val in enumerate(uncertainty_col):
    if i >20:
        uncertainty_col[i] = uncertainty_col[i]*5


In [81]:
output_matchpoints = pd.DataFrame({
    'depth1': ngrip_match_points, #ngrip
    'depth2': grip_wdc_tephra['WDC m'], #wdc
    'age_unc': uncertainty_col
    })

In [82]:
output_matchpoints.to_csv('/Users/quinnmackay/Documents/GitHub/paleochrono/BICC2025/NGRIP-WDC/iceice_synchro_horizons.txt', index=None, sep='\t')